In [78]:
import impyte
reload(impyte)
import pandas as pd
from tools.data_prep import remove_random

In [79]:
data = pd.read_csv('data/hr_test.csv')

In [80]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [81]:
data = remove_random(data, .25)

In [82]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.59,3.0,141.0,3.0,0.0,0.0,0.0,RandD,low
1,NaN,0.75,3.0,232.0,3.0,0.0,0.0,0.0,sales,low
2,NaN,0.97,5.0,202.0,2.0,0.0,0.0,0.0,IT,low
3,NaN,0.96,4.0,257.0,3.0,0.0,0.0,0.0,sales,medium
4,NaN,0.85,2.0,164.0,5.0,0.0,1.0,0.0,support,low


In [88]:
reload(impyte)
imputer = impyte.Imputer()
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()

In [89]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

The slowest run took 5.19 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.43 µs per loop
The slowest run took 4.23 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.96 µs per loop


In [72]:
# Showcase nan_check
%timeit pattern_log.row_nan_pattern(['yes', 'no', '', None])

100000 loops, best of 3: 9.31 µs per loop


In [73]:
%timeit pattern_log.print_pattern(data)

1 loop, best of 3: 800 ms per loop


In [91]:
%timeit pattern_log.pattern(data)

1 loop, best of 3: 1.81 s per loop


In [92]:
pattern_log.pattern(data)['table']

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
10,1,1,1,1,1,1,1,1,1,1,11259
8,1,1,1,1,1,1,1,1,NaN,1,374
3,1,1,1,NaN,1,1,1,1,1,1,374
2,1,1,NaN,1,1,1,1,1,1,1,374
9,1,1,1,1,1,1,1,1,1,NaN,374
7,1,1,1,1,1,1,1,NaN,1,1,374
4,1,1,1,1,NaN,1,1,1,1,1,374
0,NaN,1,1,1,1,1,1,1,1,1,374
1,1,NaN,1,1,1,1,1,1,1,1,374
5,1,1,1,1,1,NaN,1,1,1,1,374
